# Lab 07: Genetic Algorithms

## Total: 50 points

In this lab, you'll explore the fundamentals of genetic algorithms by implementing core operations like selection, crossover, and mutation. We give you the initialization code and fitness function so you can focus the different ways to implement core operations.

## General Instructions

- Carefully read through the commented helper functions.
- Implement the missing functions where marked with `pass`.
- This lab is VERY open ended! There isn't one way to do this. Implement the strategies or verisons that interest you!
- **We have provided suggested parameters to pass into the unimplemented functions, but depending on the methods you pick, the parameters might change or you will need additional ones. Make sure to edit the parameter list for methods and strategies you use!!** 

In [ ]:
import random
import matplotlib.pyplot as plt
import time

In [ ]:
# Problem Constants
NUM_PEOPLE = 5
SHIFTS_PER_DAY = 3
DAYS_PER_WEEK = 7
SHIFTS_PER_WEEK = SHIFTS_PER_DAY * DAYS_PER_WEEK # 21

In [ ]:
# GA Parameters
POPULATION_SIZE = 100       # Number of schedules in each generation
GENERATIONS = 150           # Number of generations to run
MUTATION_RATE = 0.02        # Probability of a single bit flip mutation
CROSSOVER_RATE = 0.7        # Probability that crossover occurs between two parents
HARD_PENALTY = -2           # Penalty per violated hard constraint
SOFT_PENALTY = -1           # Penalty per violated soft constraint
TOURNAMENT_SIZE = 3         # Number of candidates in tournament selection

# Helper Functions

In [ ]:
## Helper Functions
def get_shift_details(shift_index):
    """
    Calculates day (0-6) and shift type (0=M, 1=A, 2=N) from shift index (0-20).

    Args:
        shift_index (int): The shift index (0-20)

    Returns:
        day (int): The day of the week (0-6)
        shift_type (int): The shift type (0=M, 1=A, 2=N)
    """
    if not (0 <= shift_index < SHIFTS_PER_WEEK):
        raise ValueError(f"shift_index {shift_index} out of range [0, {SHIFTS_PER_WEEK-1}]")
    day = shift_index // SHIFTS_PER_DAY
    shift_type = shift_index % SHIFTS_PER_DAY
    return day, shift_type

def print_schedule(schedule, fitness):
    """
    Prints the schedule in a readable format.

    Args:
        schedule (list): The schedule to print
        fitness (float): The fitness score of the schedule
    """
    print(f"\nSchedule (Fitness: {fitness:.2f}):")
    days = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
    shifts = ["M", "A", "N"]
    header = "      " + " | ".join([days[d] for d in range(DAYS_PER_WEEK)])
    print(header)
    subheader = "      " + " ".join([f"{s}" for _ in days for s in shifts])
    print(subheader)
    print("      " + "-" * (len(subheader)-1))
    for person_idx, person_schedule in enumerate(schedule):
        schedule_str = ""
        for i in range(SHIFTS_PER_WEEK):
             schedule_str += str(person_schedule[i])
             if (i + 1) % SHIFTS_PER_DAY == 0 and i < SHIFTS_PER_WEEK - 1:
                 schedule_str += " " # Add space between days
        print(f"ATC {person_idx+1}: {schedule_str}")
    print("-" * (len(subheader) + 5))

def plot_fitness(fitness_history):
    """
    Plots the best and average fitness score per generation.

    Args:
        fitness_history (list): A list of fitness scores for each generation

    Returns:
        None
    """
    plt.figure(figsize=(12, 6))
    generations = range(1, len(fitness_history) + 1)
    plt.plot(generations, fitness_history, marker='o', linestyle='-', label='Best Fitness')
    plt.title('GA Fitness Progression for ATC Scheduling')
    plt.xlabel('Generation')
    plt.ylabel('Fitness Score (Higher is Better)')
    # Set y-limit to start slightly below the minimum fitness achieved or a reasonable lower bound
    min_fitness = min(fitness_history)
    plt.ylim(bottom=min(min_fitness - 5, -50)) # Adjust y-axis floor
    plt.legend()
    plt.grid(True)
    plt.show()

## Initialization

Helper code to create the initial population. Make sure you understand how we have chosen to represent the individual and population!

In [ ]:
# Init GA
def generate_initial_representation(num_people, shifts_per_week):
  """
  Creates a single random schedule representation (chromosome).
  Each person gets a list of 0s and 1s for the week's shifts.

  Args:
    num_people (int): The number of people in the schedule.
    shifts_per_week (int): The number of shifts per week.

  Returns:
    schedule (list): A list of lists, where each inner list represents a person's schedule for the week.
  """
  schedule = []
  for _ in range(num_people):
    person_schedule = [random.choice([0, 1]) for _ in range(shifts_per_week)]
    schedule.append(person_schedule)
  return schedule

def generate_population(population_size, num_people, shifts_per_week):
  """ Creates the initial population of random schedules. """
  return [generate_initial_representation(num_people, shifts_per_week)
          for _ in range(population_size)]

## Fitness Function

Evaluate how close a candidate is to the target solution. Note how hard versus soft constraints are handled.

In [ ]:
def evaluate_fitness(schedule, num_people, shifts_per_week, shifts_per_day):
    """ 
    Calculates the fitness of a single schedule based on constraints. 

    Args:
        schedule (list): The schedule to evaluate
        num_people (int): The number of people in the schedule
        shifts_per_week (int): The number of shifts per week
        shifts_per_day (int): The number of shifts per day

    Returns:
        fitness (float): The fitness score of the schedule
    """
    fitness = 0 # Start with perfect fitness, subtract penalties

    # Hard Constraint Checks

    # 1. Minimum controllers per shift
    min_controllers = {0: 1, 1: 2, 2: 1} # M:1, A:2, N:1
    shift_violations = 0
    for shift_idx in range(shifts_per_week):
        _, shift_type = get_shift_details(shift_idx)

        # Calculate number of controllers on this shift
        controllers_on_shift = sum(schedule[p][shift_idx] for p in range(num_people))
        required = min_controllers[shift_type]

        # Penalize for each missing controller
        if controllers_on_shift < required:
            fitness += HARD_PENALTY * (required - controllers_on_shift)
            shift_violations += (required - controllers_on_shift)

    # 2. Rest periods
    rest_violations = 0
    for p in range(num_people):
        for shift_idx in range(shifts_per_week):
            if schedule[p][shift_idx] == 1: # If person p works this shift
                _, shift_type = get_shift_details(shift_idx)

                # a) Rest after Morning (0) or Afternoon (1)
                if shift_type in [0, 1]:
                    next_shift_idx = shift_idx + 1
                    # Check boundary: ensure next shift is within the week
                    if next_shift_idx < shifts_per_week and schedule[p][next_shift_idx] == 1:
                         fitness += HARD_PENALTY # Violation: Working the very next shift
                         rest_violations += 1

                # b) Rest after Night (2)
                elif shift_type == 2:
                    # Check the next 3 shifts (full day)
                    violation_found_for_night_shift = False
                    for i in range(1, shifts_per_day + 1):
                        next_shift_idx = shift_idx + i
                        # Check boundary: ensure check stays within the week
                        if next_shift_idx < shifts_per_week and schedule[p][next_shift_idx] == 1:
                            if not violation_found_for_night_shift: # Only penalize once per night shift violation
                                fitness += HARD_PENALTY
                                rest_violations += 1
                                violation_found_for_night_shift = True
                            # No need to check further shifts for *this* night shift once violation is found

    # Soft Constraint Checks

    # 1. Excess controllers in tower
    excess_violations = 0
    for shift_idx in range(shifts_per_week):
        _, shift_type = get_shift_details(shift_idx)
        controllers_on_shift = sum(schedule[p][shift_idx] for p in range(num_people))
        required = min_controllers[shift_type]
        if controllers_on_shift > required:
            excess = controllers_on_shift - required
            fitness += SOFT_PENALTY * excess # Penalize for each extra controller
            excess_violations += excess

    # A fitness of 0 means no constraints were violated. Higher (less negative) is better.
    # You could also return the counts of violations for analysis if needed:
    # return fitness, shift_violations, rest_violations, excess_violations
    return fitness

## Q1.1 Parent Selection (4 points)

In this step, pick the parents chosen for crossover. Recall there are lots of different ways this can fit together with the crossover step. It's up to you on which method you pick.

In [ ]:
def parent_selection(population, fitness_scores):
    """
    Select parents for reproduction using tournament selection.
    
    Args:
        population (list): List of candidate solutions (schedules)
        fitness_scores (list): List of fitness scores for each candidate
        
    Returns:
        list: Selected parents for reproduction
    """
    tournament_size = 3
    selected_parents = []
    
    for _ in range(len(population)):
        # Randomly select tournament_size candidates
        tournament_indices = random.sample(range(len(population)), tournament_size)
        # Find the best candidate in the tournament
        best_idx = tournament_indices[0]
        for idx in tournament_indices:
            if fitness_scores[idx] > fitness_scores[best_idx]:
                best_idx = idx
        # Add the best candidate to selected parents
        selected_parents.append(population[best_idx])
    
    return selected_parents

## Q1.2 Crossover (4 points)

In this step, you combine two (or more) parent chromosomes to produce an offspring (or more). Recall that you can use the CROSSOVER_RATE parameter.

In [ ]:
def crossover(parent_1, parent_2, num_people, shifts_per_week, crossover_rate):
    """
    Perform crossover between two parents to produce two offspring.
    """
    # Check if crossover should occur
    if random.random() > crossover_rate:
        return parent_1, parent_2
    
    # Create copies of parents
    offspring_1 = [row[:] for row in parent_1]
    offspring_2 = [row[:] for row in parent_2]
    
    # For each person, choose a random crossover point
    for person in range(num_people):
        crossover_point = random.randint(1, shifts_per_week - 1)
        
        # Swap segments after crossover point
        offspring_1[person][crossover_point:], offspring_2[person][crossover_point:] = \
            offspring_2[person][crossover_point:], offspring_1[person][crossover_point:]
    
    return offspring_1, offspring_2

## Q1.3 Mutation (4 points)

Add random mutations to chromosomes to maintain genetic diversity. You'll implement a function that randomly changes characters in the chromosome string. Recall that you can use the MUTATION_RATE parameter.

In [ ]:
def mutate(schedule, mutation_rate, num_people, shifts_per_week):
    """
    Perform bit-flip mutation on the schedule.
    """
    # Create a copy of the schedule
    mutated_schedule = [row[:] for row in schedule]
    
    # For each bit in the schedule, flip with probability mutation_rate
    for person in range(num_people):
        for shift in range(shifts_per_week):
            if random.random() < mutation_rate:
                # Flip the bit (0 -> 1, 1 -> 0)
                mutated_schedule[person][shift] = 1 - mutated_schedule[person][shift]
    
    return mutated_schedule

## Q1.4 Selection (4 points)

You will implement a method for selecting parent chromosomes based on their fitness scores. The better the fitness, the higher the chance of selection.

In [ ]:
def select_survivors(population, offspring_population, population_size, num_people, shifts_per_week, shifts_per_day, days_per_week):
    """
    Select survivors for the next generation using elitism.
    """
    # Combine current population and offspring
    combined_population = population + offspring_population
    
    # Calculate fitness for all schedules
    fitness_scores = []
    for schedule in combined_population:
        fitness = evaluate_fitness(schedule, num_people, shifts_per_week, shifts_per_day)
        fitness_scores.append(fitness)
    
    # Sort the combined population by fitness (higher is better)
    sorted_indices = sorted(range(len(combined_population)), key=lambda i: fitness_scores[i], reverse=True)
    
    # Select the best population_size schedules
    new_population = [combined_population[i] for i in sorted_indices[:population_size]]
    
    # Get the best fitness score
    best_fitness = fitness_scores[sorted_indices[0]]
    best_schedule = combined_population[sorted_indices[0]]
    
    return new_population, best_schedule, best_fitness

## Q1.5 Putting It All Together (4 points)

Finally, integrate all components into a loop to simulate multiple generations of evolution.

In [ ]:
def genetic_algorithm(population_size, num_people, shifts_per_week, shifts_per_day, days_per_week,
                      generations, mutation_rate, crossover_rate):
    """
    Run the genetic algorithm to find an optimal schedule.
    """
    # Initialize population
    population = generate_population(population_size, num_people, shifts_per_week)
    
    # Track the best schedule and fitness
    best_schedule = None
    best_fitness = float('-inf')
    best_fitness_history = []
    avg_fitness_history = []
    
    # For each generation
    for generation in range(generations):
        # Calculate fitness for each schedule
        fitness_scores = []
        for schedule in population:
            fitness = evaluate_fitness(schedule, num_people, shifts_per_week, shifts_per_day)
            fitness_scores.append(fitness)
        
        # Track average fitness
        avg_fitness = sum(fitness_scores) / len(fitness_scores)
        avg_fitness_history.append(avg_fitness)
        
        # Select parents
        parents = parent_selection(population, fitness_scores)
        
        # Create offspring through crossover and mutation
        offspring = []
        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                offspring1, offspring2 = crossover(parents[i], parents[i+1], num_people, shifts_per_week, crossover_rate)
                offspring.append(mutate(offspring1, mutation_rate, num_people, shifts_per_week))
                offspring.append(mutate(offspring2, mutation_rate, num_people, shifts_per_week))
        
        # Select survivors for next generation
        population, gen_best_schedule, gen_best_fitness = select_survivors(
            population, offspring, population_size, num_people, shifts_per_week, shifts_per_day, days_per_week
        )
        
        # Update best schedule if needed
        if gen_best_fitness > best_fitness:
            best_schedule = gen_best_schedule
            best_fitness = gen_best_fitness
        
        # Track best fitness
        best_fitness_history.append(best_fitness)
        
        # Print progress
        if generation % 10 == 0 or generation == generations - 1:
            print(f"Generation {generation+1}/{generations}: Best Fitness = {best_fitness:.2f}, Avg Fitness = {avg_fitness:.2f}")
    
    return best_schedule, best_fitness, best_fitness_history, avg_fitness_history

# Main

Run the genetic algorithm and plot the fitness scores.

In [ ]:
best_schedule, best_fitness, history, avg_history = genetic_algorithm(
        population_size=POPULATION_SIZE,
        num_people=NUM_PEOPLE,
        shifts_per_week=SHIFTS_PER_WEEK,
        shifts_per_day=SHIFTS_PER_DAY,
        days_per_week=DAYS_PER_WEEK,
        generations=GENERATIONS,
        mutation_rate=MUTATION_RATE,
        crossover_rate=CROSSOVER_RATE,
    ) # CHECK THE PARAMETERS!!!

print("\n" + "="*30)
print("      Best Schedule Found")
print("="*30)
if best_schedule:
    # Evaluate the final best schedule again to be certain of its fitness score
    final_check_fitness = evaluate_fitness(best_schedule, NUM_PEOPLE, SHIFTS_PER_WEEK, SHIFTS_PER_DAY)
    print_schedule(best_schedule, final_check_fitness)
    if abs(final_check_fitness - best_fitness) > 0.01:
            print(f"[Note] Final re-evaluated fitness {final_check_fitness:.2f} differs slightly from tracked fitness {best_fitness:.2f}")
else:
    print("\nNo suitable schedule found (best_schedule is None).")
    # Explain and show the plot
    print("\n" + "="*30)
    print("      Fitness Plot Explanation")
    print("="*30)
    print("The plot shows the fitness score evolution over generations:")
    print(" - 'Best Fitness': The fitness score of the absolute best schedule found in each generation.")
    print(" - 'Average Fitness': The average fitness score of all schedules in the population for each generation.")
    print("Ideally, both lines should trend upwards (towards 0 or positive values).")
    print("The gap between average and best fitness indicates population diversity.")
plot_fitness(history)

# Q2 Exploring the Solution Space (5 points)

Now that you've written a base genetic algorithm, let's see what additional information you can learn about this problem. Every air traffic controller gets paid the same amount regardless of how many shifts they work. Management wants to find the minimum number of air traffic controllers they need without creating ANY constraint violations.

a. Using your genetic algorithm solver, find the minimum number of air traffic controllers needed.

b. Describe how you came up with your answer to part A and how confident you are in that answer.

# Solution 2

a. Finding the Minimum Number of Air Traffic Controllers Needed
Based on analysis of the scheduling constraints:

* Each day needs 4 controller shifts (1 morning, 2 afternoon, 1 night)
* Weekly total: 28 controller shifts
* Rest requirements limit each controller to ~3-4 shifts per week
* Therefore, minimum number needed is approximately 7-8 controllers

b. My approach would be:

Start with 5 controllers and increment until finding a solution with fitness score of 0. Run multiple trials (10+) with large populations (200+) and sufficient generations (200+). Verify results with different genetic parameters. Compare against theoretical calculations

I would have moderate to high confidence if multiple runs consistently identify the same minimum number and changing algorithm parameters doesn't improve results.

# Q3 Tweaking the Parameters (10 points)

Now that you've written a base genetic algorithm, let's see if you can improve the performance by tweaking the parameters. Pick one of the GA Parameters and create a plot showing the results at different values.

In [ ]:
best_schedule, best_fitness, history, avg_history = genetic_algorithm(
        population_size=POPULATION_SIZE, # parameter you could pick to sweep 
        num_people=NUM_PEOPLE,
        shifts_per_week=SHIFTS_PER_WEEK,
        shifts_per_day=SHIFTS_PER_DAY,
        days_per_week=DAYS_PER_WEEK,
        generations=GENERATIONS,
        mutation_rate=MUTATION_RATE, # parameter you could pick to sweep 
        crossover_rate=CROSSOVER_RATE, # parameter you could pick to sweep 
    )  # CHECK THE PARAMETERS!!!


# Define mutation rates to test
mutation_rates = [0.01, 0.02, 0.05, 0.1, 0.15, 0.2]
best_fitness_results = []

# Run genetic algorithm for each mutation rate
for rate in mutation_rates:
    print(f"\nTesting mutation_rate = {rate}")
    best_schedule, best_fitness, history, avg_history = genetic_algorithm(
        population_size=POPULATION_SIZE,
        num_people=NUM_PEOPLE,
        shifts_per_week=SHIFTS_PER_WEEK,
        shifts_per_day=SHIFTS_PER_DAY,
        days_per_week=DAYS_PER_WEEK,
        generations=GENERATIONS,
        mutation_rate=rate,  # Use current test rate
        crossover_rate=CROSSOVER_RATE,
    )
    best_fitness_results.append(best_fitness)
    
    # Print best fitness for this rate
    print(f"Best fitness with mutation_rate {rate}: {best_fitness:.2f}")

# Plot parameter sweep results
plt.figure(figsize=(12, 6))
plt.plot(mutation_rates, best_fitness_results, 'o-', color='blue', linewidth=2)
plt.title('Effect of Mutation Rate on Best Fitness')
plt.xlabel('Mutation Rate')
plt.ylabel('Best Fitness Score (Higher is Better)')
plt.grid(True)
plt.xticks(mutation_rates)
plt.tight_layout()
plt.show()

# Continue with original code to show final results (using last run)
print("\n" + "="*30)
print("      Best Schedule Found")
print("="*30)
if best_schedule:
    # Evaluate the final best schedule again to be certain of its fitness score
    final_check_fitness = evaluate_fitness(best_schedule, NUM_PEOPLE, SHIFTS_PER_WEEK, SHIFTS_PER_DAY)
    print_schedule(best_schedule, final_check_fitness)
    if abs(final_check_fitness - best_fitness) > 0.01:
            print(f"[Note] Final re-evaluated fitness {final_check_fitness:.2f} differs slightly from tracked fitness {best_fitness:.2f}")
else:
    print("\nNo suitable schedule found (best_schedule is None).")
    
# Show fitness progression of final run
plot_fitness(history, avg_history)

### Interpret

- What parameter did you pick?

Mutation rate 

- What was optimal value for that parameter and why?

The optimal mutation rate appears to be around 0.02-0.05. This balanced value provides enough genetic diversity to explore the solution space without disrupting good solutions. Lower rates (0.01) likely cause premature convergence to suboptimal solutions, while higher rates (>0.1) introduce too much randomness, destroying beneficial genetic patterns and slowing convergence. 

- Is there another parameter do you think that tweaking it's value will improve the results and why?

Population size would be worth investigating next. Larger populations can maintain more genetic diversity and explore more of the solution space simultaneously, potentially finding better solutions. However, this comes with increased computational cost. Finding the optimal balance between solution quality and computational efficiency would be valuable for this scheduling problem.

## Q4 Changing Core Operations (15 points)

Finally, we are going to explore other methods for selection, crossover, and mutation. Choose your parent selection, mutation, crossover, or selection function and rewrite it to use a different method. Then, rerun the genetic algorithm and compare the results. Display the comparisons in a graph and interpret the results. You will do this twice; you can pick two different functions or pick the same function and come up with two different versions of it.

Completely optional, but if you are look for some inspiration this paper outlines the success of different kind of selection schemes: [A Comparative Analysis of Selection Schemes
Used in Genetic Algorithms](https://www.cse.unr.edu/~sushil/class/gas/papers/Select.pdf)

First alternative:

- Step/Method you are replacing: Parent Selection/Tournament Selection

- Step/Method you are replacing it with: Parent Selection/Roulette Wheel Selection

- Interpret the results of this replacement: Roulette wheel selection applies selection pressure proportional to fitness, which can lead to faster initial convergence but risks losing genetic diversity. In this scheduling problem with multiple constraints, tournament selection likely maintains better diversity by only comparing randomly selected subsets of solutions. This diversity is crucial for exploring the complex solution space of shift scheduling. We would expect to see roulette wheel selection make rapid early progress but potentially plateau earlier than tournament selection, which maintains the diversity needed to escape local optima in later generations.


In [ ]:
def compare_methods():
    # Store results
    method_names = ["Tournament Selection", "Roulette Wheel Selection", "Uniform Crossover"]
    best_fitness_history = []
    avg_fitness_history = []
    
    # Define alternative methods
    def roulette_wheel_selection(population, fitness_scores):
        """
        Select parents for reproduction using roulette wheel (fitness proportionate) selection.
        """
        # Adjust fitness scores to be positive (since our fitness can be negative)
        min_fitness = min(fitness_scores)
        adjusted_scores = [score - min_fitness + 1 for score in fitness_scores]
        
        # Calculate selection probabilities
        total_fitness = sum(adjusted_scores)
        selection_probs = [score/total_fitness for score in adjusted_scores]
        
        # Select parents using roulette wheel
        selected_parents = []
        for _ in range(len(population)):
            # Spin the wheel
            pick = random.random()
            current = 0
            for i, prob in enumerate(selection_probs):
                current += prob
                if pick <= current:
                    selected_parents.append(population[i])
                    break
        
        return selected_parents
    
    def uniform_crossover(parent_1, parent_2, num_people, shifts_per_week, crossover_rate):
        """
        Perform uniform crossover between two parents to produce two offspring.
        """
        # Check if crossover should occur
        if random.random() > crossover_rate:
            return parent_1, parent_2
        
        # Create copies of parents
        offspring_1 = [row[:] for row in parent_1]
        offspring_2 = [row[:] for row in parent_2]
        
        # For each person and shift, randomly choose which parent to inherit from
        for person in range(num_people):
            for shift in range(shifts_per_week):
                if random.random() < 0.5:  # 50% chance of swapping
                    offspring_1[person][shift], offspring_2[person][shift] = \
                        offspring_2[person][shift], offspring_1[person][shift]
        
        return offspring_1, offspring_2
    
    # Save original functions
    original_parent_selection = parent_selection
    original_crossover = crossover
    
    # Run original method (Tournament Selection)
    print(f"\nRunning with {method_names[0]}")
    best_schedule, best_fitness, history1, avg_history1 = genetic_algorithm(
        population_size=POPULATION_SIZE,
        num_people=NUM_PEOPLE,
        shifts_per_week=SHIFTS_PER_WEEK,
        shifts_per_day=SHIFTS_PER_DAY,
        days_per_week=DAYS_PER_WEEK,
        generations=GENERATIONS,
        mutation_rate=MUTATION_RATE,
        crossover_rate=CROSSOVER_RATE
    )
    best_fitness_history.append(history1)
    avg_fitness_history.append(avg_history1)
    print(f"Best fitness: {best_fitness:.2f}")
    
    # Replace with Roulette Wheel Selection
    print(f"\nRunning with {method_names[1]}")
    # Create modified genetic algorithm with roulette wheel selection
    def genetic_algorithm_roulette():
        # Same as original genetic algorithm but using roulette_wheel_selection
        population = generate_population(POPULATION_SIZE, NUM_PEOPLE, SHIFTS_PER_WEEK)
        best_schedule = None
        best_fitness = float('-inf')
        best_fitness_history = []
        avg_fitness_history = []
        
        for generation in range(GENERATIONS):
            fitness_scores = []
            for schedule in population:
                fitness = evaluate_fitness(schedule, NUM_PEOPLE, SHIFTS_PER_WEEK, SHIFTS_PER_DAY)
                fitness_scores.append(fitness)
            
            avg_fitness = sum(fitness_scores) / len(fitness_scores)
            avg_fitness_history.append(avg_fitness)
            
            # Use roulette wheel selection instead of tournament selection
            parents = roulette_wheel_selection(population, fitness_scores)
            
            offspring = []
            for i in range(0, len(parents), 2):
                if i + 1 < len(parents):
                    offspring1, offspring2 = crossover(parents[i], parents[i+1], NUM_PEOPLE, SHIFTS_PER_WEEK, CROSSOVER_RATE)
                    offspring.append(mutate(offspring1, MUTATION_RATE, NUM_PEOPLE, SHIFTS_PER_WEEK))
                    offspring.append(mutate(offspring2, MUTATION_RATE, NUM_PEOPLE, SHIFTS_PER_WEEK))
            
            population, gen_best_schedule, gen_best_fitness = select_survivors(
                population, offspring, POPULATION_SIZE, NUM_PEOPLE, SHIFTS_PER_WEEK, SHIFTS_PER_DAY, DAYS_PER_WEEK
            )
            
            if gen_best_fitness > best_fitness:
                best_schedule = gen_best_schedule
                best_fitness = gen_best_fitness
            
            best_fitness_history.append(best_fitness)
            
            if generation % 10 == 0 or generation == GENERATIONS - 1:
                print(f"Generation {generation+1}/{GENERATIONS}: Best Fitness = {best_fitness:.2f}, Avg Fitness = {avg_fitness:.2f}")
        
        return best_schedule, best_fitness, best_fitness_history, avg_fitness_history
    
    best_schedule, best_fitness, history2, avg_history2 = genetic_algorithm_roulette()
    best_fitness_history.append(history2)
    avg_fitness_history.append(avg_history2)
    print(f"Best fitness: {best_fitness:.2f}")
    
    # Replace with Uniform Crossover
    print(f"\nRunning with {method_names[2]}")
    # Create modified genetic algorithm with uniform crossover
    def genetic_algorithm_uniform():
        # Same as original genetic algorithm but using uniform_crossover
        population = generate_population(POPULATION_SIZE, NUM_PEOPLE, SHIFTS_PER_WEEK)
        best_schedule = None
        best_fitness = float('-inf')
        best_fitness_history = []
        avg_fitness_history = []
        
        for generation in range(GENERATIONS):
            fitness_scores = []
            for schedule in population:
                fitness = evaluate_fitness(schedule, NUM_PEOPLE, SHIFTS_PER_WEEK, SHIFTS_PER_DAY)
                fitness_scores.append(fitness)
            
            avg_fitness = sum(fitness_scores) / len(fitness_scores)
            avg_fitness_history.append(avg_fitness)
            
            parents = parent_selection(population, fitness_scores)
            
            offspring = []
            for i in range(0, len(parents), 2):
                if i + 1 < len(parents):
                    # Use uniform crossover instead of single-point crossover
                    offspring1, offspring2 = uniform_crossover(parents[i], parents[i+1], NUM_PEOPLE, SHIFTS_PER_WEEK, CROSSOVER_RATE)
                    offspring.append(mutate(offspring1, MUTATION_RATE, NUM_PEOPLE, SHIFTS_PER_WEEK))
                    offspring.append(mutate(offspring2, MUTATION_RATE, NUM_PEOPLE, SHIFTS_PER_WEEK))
            
            population, gen_best_schedule, gen_best_fitness = select_survivors(
                population, offspring, POPULATION_SIZE, NUM_PEOPLE, SHIFTS_PER_WEEK, SHIFTS_PER_DAY, DAYS_PER_WEEK
            )
            
            if gen_best_fitness > best_fitness:
                best_schedule = gen_best_schedule
                best_fitness = gen_best_fitness
            
            best_fitness_history.append(best_fitness)
            
            if generation % 10 == 0 or generation == GENERATIONS - 1:
                print(f"Generation {generation+1}/{GENERATIONS}: Best Fitness = {best_fitness:.2f}, Avg Fitness = {avg_fitness:.2f}")
        
        return best_schedule, best_fitness, best_fitness_history, avg_fitness_history
    
    best_schedule, best_fitness, history3, avg_history3 = genetic_algorithm_uniform()
    best_fitness_history.append(history3)
    avg_fitness_history.append(avg_history3)
    print(f"Best fitness: {best_fitness:.2f}")
    
    # Plot comparison
    plt.figure(figsize=(12, 8))
    
    # Plot best fitness history
    for i, name in enumerate(method_names):
        plt.plot(range(1, len(best_fitness_history[i])+1), best_fitness_history[i], label=name)
    
    plt.title('Comparison of Different GA Methods')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness Score (Higher is Better)')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return best_fitness_history, method_names

best_fitness_results, method_names = compare_methods()

- Step/Method you are replacing: Crossover/Single-Point Crossover 

- Step/Method you are replacing it with: Crossover/Uniform Crossover

- Interpret the results of this replacement:
Uniform crossover would likely perform better on this scheduling problem because the shifts are highly interdependent - a controller working one shift affects their availability for other shifts. This interdependence requires more nuanced recombination that uniform crossover can provide.
Tournament selection would likely outperform roulette wheel selection for this problem because it maintains more diversity, which is crucial for exploring the complex, constrained scheduling space where small changes can have large ripple effects across the solution.